In [1]:
import keras
import pickle
from keras.utils import np_utils
import numpy as np
from tqdm import tqdm_notebook as tqdm

/home/parubchenko/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [40]:
import glob

In [179]:
melodies = []
with open('my_data/notes', 'rb') as filepath:
    melodies = pickle.load(filepath)

In [180]:
labels = []
with open('my_data/labels', 'rb') as filepath:
    labels = pickle.load(filepath)

In [181]:
notes = []

for melody in tqdm(melodies):
    key = melody[0]
    if key in labels:
        notes.append('Start')
        for note in melody[1]:
            if '.' not in note or len(note.split('.')) <= 3:
                notes.append(note)
        notes.append('End')

In [182]:
# Get all pitch names
pitchnames = sorted(set(item for item in notes))
# Get all pitch names
n_vocab = len(set(notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

In [184]:
MODEL = 10 # 0, 1, 2, 3, 10

In [235]:
if MODEL == 10:
    sequence_length = 32
else:
    sequence_length = 100

network_input = []
emotion_input = []
network_output = []
from collections import Counter
types = Counter()
summa = 0
for melody in tqdm(melodies):
    key = melody[0]
    if key in labels:
        emotion = labels[key]
        melody_notes = ['Start'] 
        for note in melody[1]:
            if '.' not in note or len(note.split('.')) <= 3:
                melody_notes.append(note)
        melody_notes += ['End']
        summa += len(melody_notes) - sequence_length
        for i in range(0, len(melody_notes) - sequence_length, 1):
            types[emotion] += 1
            sequence_in = melody_notes[i:i + sequence_length]
            network_input.append([note_to_int[char] for char in sequence_in])
            emotion_input.append(emotion)
            if MODEL < 10:
                sequence_out = melody_notes[i + sequence_length]
                network_output.append(note_to_int[sequence_out])
            elif MODEL == 10:
                sequence_out = melody_notes[i + 1: i + sequence_length + 1]
                network_output.append([note_to_int[char] for char in sequence_out])
print(summa)

275361


In [191]:
if MODEL == 0:
    n_patterns = len(network_input)
    # reshape the input into a format compatible with LSTM layers
    Songs = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    Songs = Songs / (float(n_vocab) - 1)
    Emotions = np_utils.to_categorical(emotion_input)
    Outputs = np_utils.to_categorical(network_output)

In [192]:
if MODEL > 0 and MODEL < 10:
    n_patterns = len(network_input)
    # reshape the input into a format compatible with LSTM layers
    Songs = np.array(network_input)
    Emotions = np_utils.to_categorical(emotion_input)
    Outputs = np_utils.to_categorical(network_output)

In [245]:
if MODEL >= 10:
    n_patterns = len(network_input)
    # reshape the input into a format compatible with LSTM layers
    Songs = np.array(network_input)
    Emotions = np_utils.to_categorical(emotion_input)
    #Outputs = np.array(network_output) / (float(n_vocab) - 1)

In [247]:
Outputs = np.ones(len(network_output))
for index, output in tqdm(enumerate(network_output)):
    Outputs[index] = np_utils.to_categorical(output)

ValueError: setting an array element with a sequence.

In [246]:
Songs.shape

(275374, 32)

In [197]:
n_emots = Emotions.shape[1]

## Many-to-One LSTM

### 0: Basic model with LSTM and Adding emotion

In [24]:
from keras.models import Model
from keras.layers import LSTM, Dropout, Dense, Activation, Input, concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [30]:
seq = Input(shape=(Songs.shape[1], Songs.shape[2]))

lstm11 = LSTM(512, return_sequences=True)(seq)
lstm12 = Dropout(0.3)(lstm11)

lstm21 = LSTM(512, return_sequences=True)(lstm12)
lstm22 = Dropout(0.3)(lstm21)

lstm31 = LSTM(256)(lstm22)
emotion = Input(shape=(Emotions.shape[1], ))

union = concatenate([lstm31, emotion])

dense11 = Dense(256)(union)
dense12 = Dropout(0.3)(dense11)

dense2 = Dense(n_vocab - 1)(dense12)
output = Activation('softmax')(dense2)

model_0 = Model(inputs=[seq, emotion], outputs=[output])
model_0.compile(loss='categorical_crossentropy', optimizer='rmsprop')

### 1: LSTM with notes embedding

In [58]:
from keras.layers import Embedding

In [71]:
seq = Input(shape=(Songs.shape[1], ))

note_emb = Embedding(input_dim=n_vocab, output_dim=3)(seq)

lstm11 = LSTM(512, return_sequences=True)(note_emb)
lstm12 = Dropout(0.3)(lstm11)

lstm21 = LSTM(512, return_sequences=True)(lstm12)
lstm22 = Dropout(0.3)(lstm21)

lstm31 = LSTM(256)(lstm22)
emotion = Input(shape=(Emotions.shape[1], ))

union = concatenate([lstm31, emotion])

dense11 = Dense(256)(union)
dense12 = Dropout(0.3)(dense11)

dense2 = Dense(n_vocab - 1)(dense12)
output = Activation('softmax')(dense2)

model_emb = Model(inputs=[seq, emotion], outputs=[output])
model_emb.compile(loss='categorical_crossentropy', optimizer='rmsprop')

### 2: LSTM + Emotions + Recover emotion

In [100]:
seq = Input(shape=(Songs.shape[1], ))

note_emb = Embedding(input_dim=n_vocab, output_dim=3)(seq)

lstm11 = LSTM(512, return_sequences=True)(note_emb)
lstm12 = Dropout(0.3)(lstm11)

lstm21 = LSTM(512, return_sequences=True)(lstm12)
lstm22 = Dropout(0.3)(lstm21)

lstm31 = LSTM(256)(lstm22)
emotion = Input(shape=(Emotions.shape[1], ))

union = concatenate([lstm31, emotion])

dense11 = Dense(256)(union)
dense12 = Dropout(0.3)(dense11)

dense_note = Dense(n_vocab - 1)(dense12)
output_note = Activation('softmax')(dense_note)

dense_emot = Dense(n_emots)(dense12)
output_emot = Activation('softmax')(dense_emot)

model_2 = Model(inputs=[seq, emotion], outputs=[output_note, output_emot])
model_2.compile(loss='categorical_crossentropy', 
                  optimizer='rmsprop')

### 3: LSTM with Emotions input

In [136]:
from keras.layers import RepeatVector

In [137]:
seq = Input(shape=(Songs.shape[1], ))

note_emb = Embedding(input_dim=n_vocab, output_dim=3)(seq)
emotion = Input(shape=(Emotions.shape[1], ))

union = concatenate([note_emb, RepeatVector(Songs.shape[1])(emotion)])

lstm11 = LSTM(512, return_sequences=True)(union)
lstm12 = Dropout(0.3)(lstm11)

lstm21 = LSTM(512, return_sequences=True)(lstm12)
lstm22 = Dropout(0.3)(lstm21)

lstm31 = LSTM(256)(lstm22)

dense11 = Dense(256)(lstm31)
dense12 = Dropout(0.3)(dense11)

dense_note = Dense(n_vocab - 1)(dense12)
output_note = Activation('softmax')(dense_note)

dense_emot = Dense(n_emots)(dense12)
output_emot = Activation('softmax')(dense_emot)

model_3 = Model(inputs=[seq, emotion], outputs=[output_note, output_emot])
model_3.compile(loss='categorical_crossentropy', 
                  optimizer='rmsprop')

In [225]:
from sklearn.model_selection import train_test_split

Song_train, Song_val, Emotion_train, Emotion_val, Out_train, Out_val = train_test_split(
    Songs, Emotions, Outputs, test_size=0.1, shuffle=True)

In [227]:
if MODEL == 0:
    model = model_0
elif MODEL == 1:
    model = model_emb
elif MODEL == 2:
    model = model_2
elif MODEL == 3:
    model = model_3

filepath = "weights/model_{}.hdf5".format(MODEL)   
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')
checkpoint = ModelCheckpoint(
    filepath, monitor='val_loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint, early_stop]     
model.fit([Song_train, Emotion_train], [Out_train, Emotion_train], 
          validation_data=([Song_val, Emotion_val], [Out_val, Emotion_val]), 
          epochs=30, batch_size=512, callbacks=callbacks_list)

Train on 235657 samples, validate on 26185 samples
Epoch 1/30
235657/235657 [==============================] - 217s 919us/step - loss: 0.2967 - dense_46_loss: 0.1780 - activation_22_loss: 0.1187 - val_loss: 0.0626 - val_dense_46_loss: 0.0613 - val_activation_22_loss: 0.0013
Epoch 2/30
235657/235657 [==============================] - 215s 910us/step - loss: 0.0907 - dense_46_loss: 0.0724 - activation_22_loss: 0.0183 - val_loss: 1.3750 - val_dense_46_loss: 0.0546 - val_activation_22_loss: 1.3204
Epoch 3/30
235657/235657 [==============================] - 212s 899us/step - loss: 0.0482 - dense_46_loss: 0.0421 - activation_22_loss: 0.0061 - val_loss: 0.0154 - val_dense_46_loss: 0.0154 - val_activation_22_loss: 9.2738e-06
Epoch 4/30
 64000/235657 [=======>......................] - ETA: 2:31 - loss: 0.0305 - dense_46_loss: 0.0296 - activation_22_loss: 8.7770e-04

KeyboardInterrupt: 

In [141]:
start = np.random.randint(0, len(Songs)-1)

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = Songs[start]

In [154]:
pattern = [note_to_int[char] for char in test_song[:100]]#test_song[0:100]

In [156]:
outputs = {}
emotions = {'sad': [1, 0, 0], 'energetic': [0, 1, 0], 'romantic': [0, 0, 1]}

for label, emotion in emotions.items(): 
    prediction_output = []
    # generate 500 notes
    for note_index in tqdm(range(500)):
        if MODEL == 0:
            prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        elif MODEL > 0:
            prediction_input = np.reshape(pattern, (1, len(pattern)))
        emotion_input = np.reshape(emotion, (1, len(emotion)))

        prediction = model.predict([prediction_input, emotion_input], verbose=0)
        if MODEL < 2:
            index = np.argmax(prediction)
        if MODEL == 2 or MODEL == 3:
            print(np.argmax(prediction[1]), end=' ')
            index = np.argmax(prediction[0])
        result = int_to_note[index]
        if result == 'End':
            break
            
        prediction_output.append(result)
        pattern = np.append(pattern, index / (float(n_vocab) - 1))
        pattern = pattern[1:len(pattern)]
    outputs[label] = prediction_output

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 

In [157]:
from music21 import converter, instrument, note, chord, stream

In [158]:
output_songs = {}

for label, prediction_output in outputs.items():
    offset = 0
    output_notes = []
    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.5
    output_songs[label] = output_notes

In [159]:
for label, output_notes in output_songs.items():
    midi_stream = stream.Stream(output_notes)
    print(label)
    midi_stream.write('midi', fp='{}_31.mid'.format(label))
    midi_stream.show('midi')

sad


energetic


romantic


In [178]:
midi = converter.parse('energetic_31.mid')
midi.show('text')

{0.0} <music21.stream.Part 0x7fdfd9e17c88>
    {0.0} <music21.chord.Chord A4 C4>
    {0.5} <music21.note.Rest rest>
    {1.0} <music21.chord.Chord A4 C4>
    {1.5} <music21.chord.Chord A4 C4>
    {2.0} <music21.chord.Chord A4 C4>
    {2.5} <music21.chord.Chord A4 C4>
    {3.0} <music21.chord.Chord A4 C4>
    {3.5} <music21.chord.Chord A4 C4>
    {4.0} <music21.chord.Chord A4 C4>
    {4.5} <music21.chord.Chord A4 C4>
    {5.0} <music21.chord.Chord A4 C4>
    {5.5} <music21.chord.Chord A4 C4>
    {6.0} <music21.chord.Chord A4 C4>
    {6.5} <music21.chord.Chord A4 C4>
    {7.0} <music21.chord.Chord A4 C4>
    {7.5} <music21.chord.Chord A4 C4>
    {8.0} <music21.chord.Chord A4 C4>
    {8.5} <music21.chord.Chord A4 C4>
    {9.0} <music21.chord.Chord A4 C4>
    {9.5} <music21.chord.Chord A4 C4>
    {10.0} <music21.chord.Chord A4 C4>
    {10.5} <music21.chord.Chord A4 C4>
    {11.0} <music21.chord.Chord A4 C4>
    {11.5} <music21.chord.Chord A4 C4>
    {12.0} <music21.chord.Chord A4 C4>
    {